# Прв парцијален испит по Основи на Вештачката Интелигенција (2/2)

Датум: 21.11.2022

## Задача 2 - Државни Патишта

Ја разгледуваме задачата со државните патишта, од аудиториските вежби. Таму проблемот се решава со пребарување со униформна цена, а потоа со пребарување А* користејќи евклидово растојание.

### Прашања
1. **(15 поени)** Кој е најдобар начин да дознаеме дали навистина алгоритамот А* побрзо стигнува до целта за разлика од пребарувањето со униформна цена? Користејќи најмалку 3 примери, покажете дека алгоритамот А* навистина работи побрзо.
1. **(10 поени)** За функцијата за дојава искористете го растојанието Менхетен. Дали алгоритамот ќе стигне до оптималното решение? Ако не, зошто не? Покажете преку пример.

1.
* најдобро е преку број на разгранети јазли

In [1]:
import pandas as pd
from plotly import graph_objects as go

In [2]:
class WeightedGraph:
    def __init__(self):
        self.graph_dict = {}
    
    def add_vertex(self, vertex):
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = {}
    
    def vertices(self):
        return list(self.graph_dict.keys())
    
    def add_edge(self, edge, add_reversed=True):
        vertex1, vertex2, weight = edge
        self.graph_dict[vertex1][vertex2] = weight
        if add_reversed:
            self.graph_dict[vertex2][vertex1] = weight
    
    def edges(self):
        edges = []
        for vertex in self.graph_dict:
            for neighbour, weight in self.graph_dict[vertex].items():
                edges.append((vertex, neighbour, weight))
        return edges
    
    def neighbours(self, vertex):
        return list(self.graph_dict[vertex].items())

In [3]:
g = WeightedGraph()
files = pd.read_csv('roads/roads_info.csv')['Кратенка']
for file in files:
    df = pd.read_csv(f'roads/{file}.csv')
    df['ОД'] = df['ЈАЗОЛ НА ПОЧЕТОКОТ']
    df['ДО'] = df['ЈАЗОЛ НА КРАЈОТ']
    for row in df.itertuples():
        g.add_vertex(row.ОД)
        g.add_vertex(row.ДО)
        g.add_edge((row.ОД, row.ДО, row.ДОЛЖИНА))

In [4]:
def city(graph,city):
    for vertex in graph.vertices():
        if city.lower() in vertex.lower():
            print(vertex)

In [5]:
city(g, 'Пробиштип')
city(g, 'Нагоричане')

364 - Пробиштип
195 - Младо Нагоричане
378 - Старо Нагоричане 1
512 - Младо Нагоричане


In [6]:
# so uniformna cena
import heapq
def uniform_cost_search(graph,starting_vertex, goal_vertex):
    global c
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        c+=1
        weight, vertex_list = heapq.heappop(queue) 
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return weight, vertex_list
        if vertex_to_expand not in expanded:
            for  neighbour, new_weight in graph.neighbours(vertex_to_expand):
                if neighbour not in expanded: 
                    heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [7]:
df = pd.read_csv('roads/coordinates.csv')

def eucledian_distance(vertex1, vertex2):
    vertex1 = int(vertex1.split('-')[0])
    vertex2 = int(vertex2.split('-')[0])
    vertex1 = df.query(f'Јазол == {vertex1}')
    vertex2 = df.query(f'Јазол == {vertex2}')
    import math
    delta_x_squared = math.pow(float(vertex1.X) - float(vertex2.X), 2)
    delta_y_squared = math.pow(float(vertex1.Y) - float(vertex2.Y), 2)
    return math.sqrt(delta_x_squared + delta_y_squared)

eucledian_distance('364 - Пробиштип', '378 - Старо Нагоричане 1')

36229.390811682366

In [8]:
def a_star_search(graph, starting_vertex, goal_vertex, alpha=1):
    global c
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        c+=1
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return current_path_weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in graph.neighbours(vertex_to_expand):
                if neighbour not in expanded:
                    path_weight = current_path_weight + new_weight
                    a_star_weight = path_weight + alpha * eucledian_distance(neighbour, goal_vertex)
                    heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [67]:
c=0
uniform_cost_search(g,'364 - Пробиштип', '378 - Старо Нагоричане 1')
c

57

In [65]:
c=0
a_star_search(g,'364 - Пробиштип', '378 - Старо Нагоричане 1')
c

12

In [68]:
c=0
uniform_cost_search(g,'183 - Инджиково (автопат)', '556 - Петровец (автопат)')
c

2

In [69]:
c=0
a_star_search(g,'183 - Инджиково (автопат)', '556 - Петровец (автопат)')
c

2

In [70]:
c=0
uniform_cost_search(g,'15 - Бучиште', '367 - Пишица')
c

4

In [71]:
c=0
a_star_search(g,'15 - Бучиште', '367 - Пишица')
c

2

b) не, нема да стигне до оптимално решение затоа што менхетен генерално се користи за пресметки каде што патот е всушност grid како во шах, и дава по ефективни чисти пресметки
* еквалидовото расотојание пак се користи за да се пресмета разликата од некој вектор и почетниот вектор, што дава малку по реална слика за патот кој ние го мериме

In [9]:
# sega so menheten
import math
df = pd.read_csv('roads/coordinates.csv')
def manhattan_distance(vertex1, vertex2):
    vertex1 = int(vertex1.split('-')[0])
    vertex2 = int(vertex2.split('-')[0])
    vertex1 = df.query(f'Јазол == {vertex1}')
    vertex2 = df.query(f'Јазол == {vertex2}')
    return abs(float(vertex1.X) - float(vertex2.X)) + abs(float(vertex1.Y) - float(vertex1.Y))
manhattan_distance('364 - Пробиштип', '378 - Старо Нагоричане 1')

29499.83999999985

In [10]:
def a_star_search(graph, starting_vertex, goal_vertex, alpha=1):
    global c
    if starting_vertex == goal_vertex:
        return []
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        c+=1
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand == goal_vertex:
            return current_path_weight, vertex_list
        if vertex_to_expand not in expanded:
            for neighbour, new_weight in graph.neighbours(vertex_to_expand):
                if neighbour not in expanded:
                    path_weight = current_path_weight + new_weight
                    a_star_weight = path_weight + alpha * manhattan_distance(neighbour, goal_vertex)
                    heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [11]:
c=0
a_star_search(g,'364 - Пробиштип', '378 - Старо Нагоричане 1')
c

17

* со еквалид беа разгранети 12 јазли
* со менхетен беа разгранети 17 јазли 